In [1]:
import re
import string
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from collections import Counter

import spacy
from spacy.cli.download import download
download(model="ru_core_news_sm")

In [4]:
df = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

## text cleaninig

In [5]:
# check for duplicates using groupby
df_nondupes = df.groupby(['text']).nunique().sort_values(by='category', ascending=False)
# find duplicates with target > 1 as a way of flagging if they are duplicate
df_dupes = df_nondupes[df_nondupes['category'] > 1]
df_dupes.rename(columns={'id':'# of duplicates', 'category':'sum of target var'})
df_dupes.sort_values('text')

,oid,category
text,,
30 дней Okko бесплатно 33 Подключайтесь и смотрите популярные фильмы и сериалы в отличном качестве с доступом на 5 устройств,2,2
60 дней Плюса Мульти в подарок по промокоду MYMUSIC 33 Если у вас когда то была подписка Плюс для вас он тоже сработает. Получите миллионы треков аудиокниги и топовые рекомендации которые подойдут именно вам 33,4,3
60 дней Плюса Мульти в подарок по промокоду MYMUSIC 33 Если у вас когда то была подписка Плюс для вас он тоже сработает. Получите тысячи треков аудиокниги и топовые рекомендации которые подойдут именно вам 33,7,4
Python и Java изучаем языки программирования на практике 12 13 мая в 19. 00 ждем вас на практическом интенсиве по языкам Python и Java За два вечера изучим основы и напишем пару проектов которые сможете показывать работодателям. Научим даже тех кто никогда не прикасался к коду Регистрация по ссылке На интенсиве Расскажем чем занимаются Python разработчики Почему Python подойдет новичкам Как использовать полученные знания Отработаем навыки на тренажере Кто такой Java разработчик и чем он занимается. Какие навыки потребуются для старта в разработке. Как развивается карьера разработчиков почему их зарплаты растут так быстро. Пишем код и пробуем простейшее программирование. Регистрация по ссылке Бонусы для участников. Спикеры Илья Сазонов Руководитель направления tokenoid разработки во Всегда. Да Анастасия Борнева Руководитель направления по исследованию данных в Сбере. Скорее регистрируйтесь и встретимся в прямом эфире,2,2
Админ здравствуйте. Пропустите или нет – не знаю так как пост спорный конечно. Мой маленький сын уже больше 2х лет на химиотерапии. Онкология был рецидив. Лечимся в Москве сами из провинции. Никаких сборов не ведем так как лечимся по страховке. Не просим никакой помощи. Но и бесплатное лечение совсем не бесплатное кто лечился – знает. Не говоря об оплате жилья транспортных расходах и сопутствующих. На лечение хватает заработанного самостоятельно но есть проблема в работе. Сына в дет сад не отдашь – опасная среда для его иммунитета внутри имплантирован порт для вливания химии. Работаем из дома – выжигаем картины на дереве. Прошу опубликовать пост – уверена среди ваших подписчиков найдется человек ищущий подарок близким тем более я не беру предоплату. Если не хотите опубликовать бесплатно могу предложить выжечь в качестве оплаты рекламы вам портрет. Документы о лечении ссылку на группу поддержки сына скину в ЛС по вашему запросу. Спасибо.,3,2
...,...,...
﻿ ️Шутки в сторону 33 Сергей Кургинян о мобилизации и русском народе Ситуация когда Запад использует все возможности для того чтобы увеличить военный потенциал Украины когда Европа и США ведут с Россией экономическую и информационную войну является для нашей страны экзистенциальной. Выстоит ли Россия или Запад осуществит давнюю мечту и покончит с нею окончательно и бесповоротно? ️Сложившаяся ситуация взыскует мобилизации русского народа. Но в ходе спецоперации от добровольцев отбоя не было? Разве нет позорных толп молодежи бегущей из России в соседние страны? Разве нет провала под Харьковом где России пришлось оставить доверившихся нам людей? На вопросы о том почему не происходит подлинной мобилизации в российском обществе и российской элите и что нужно делать для того чтобы она началась ответил ⭐️ политолог философ и лидер движения Суть времени Сергей Кургинян в программе Сергея Мардана на Радио Комсомольская правда. ️По словам Кургиняна овцы не могут осуществить мобилизацию против волков. Справиться с этой задачей может лишь волкодав. Значит нужно убрать тех людей которые саботируют мобилизацию которые к ней не способны и найти людей совсем другого склада 33 Шутки кончились речь о выживании страны. . . Мобилизация ВСРФ Россия Кургинян Донбасс Война СВО Украина УГИЛ,2,2
﻿Американский педиатр разгромила гендерную теорию Кретелла врач с двадцатилетним стажем рассказывает о том почему трансгендерная идеология неверна. Врач рассказывает почему биологический пол невозможно назначить. Пол определяется

In [6]:
df = df[~df.text.isin(df_dupes.reset_index().text)]
df_test = df_test[~df_test.text.isin(df_dupes.reset_index().text)]

In [7]:
#tokenization
tok = spacy.load('ru_core_news_sm')
def tokenize (text):
    # print(text)
    # text = re.sub(r"[^\x00-\x7F]+", " ", text)
    # print(text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]



In [8]:
#count number of occurences of each word
counts = Counter()
for index, row in df.iterrows():
    counts.update(tokenize(row['text']))
for index, row in df_test.iterrows():
    counts.update(tokenize(row['text']))

In [10]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words before: 182169
num_words after: 107652


In [9]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [12]:
df['text'].apply(lambda x: len(x.split(' '))).mean()

60.9300149015712

In [10]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [11]:
df['encoded'] = df['text'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
df.head()

/tmp/ipykernel_17964/2447743987.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df['encoded'] = df['text'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))


,oid,category,text,encoded
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...,"[[2, 3, 4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...,"[[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 12, 9,..."
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...,"[[45, 46, 47, 48, 49, 50, 51, 15, 52, 53, 54, ..."
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...,"[[129, 130, 24, 131, 132, 103, 133, 134, 33, 1..."
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...,"[[262, 263, 264, 9, 265, 266, 267, 12, 268, 9,..."


## Train

In [12]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [13]:
class TestDataset(Dataset):
    def __init__(self, X, ido):
        self.X = X
        self.ido = ido
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.ido[idx], torch.from_numpy(self.X[idx][0].astype(np.int32)), self.X[idx][1]

In [14]:
labels = dict(zip(df.category.unique(), list(range(len(df.category.unique())))))
# Prepare columns
df['label'] = df.category.map(labels.get)

In [15]:
X = df['encoded'].values
Y = df['label'].values

X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

In [16]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [17]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=64, shuffle=False)

In [18]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long().to(device)
            y = y.long().to(device)
            l = l.to(device)
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = nn.functional.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long().to(device)
        y = y.long().to(device)
        l = l.to(device)
        y_hat = model(x, l)
        loss = nn.functional.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred.cpu(), y.cpu().unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [24]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=2)
        self.linear = nn.Linear(hidden_dim, 13)
        self.dropout = nn.Dropout(0.4)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [25]:
device = 'cuda'

In [ ]:
model_fixed = LSTM_fixed_len(len(vocab2index), 150, 70)
model_fixed.to(device)
train_model(model_fixed, epochs=30, lr=0.01)

## predict

In [ ]:
test_df = pd.read_csv('test.csv')
test_df

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...
...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...
26256,279369911,Волейбол от первого лица Егора Пупынина переко...
26257,600699419,Вы были когда нибудь на свидании где вам задав...
26258,560223506,ТОП 20 самых эффективных общефизических упражн...


In [ ]:
test_df['encoded'] = test_df['text'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
test_df.head()

<ipython-input-157-076a980d623b>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_df['encoded'] = test_df['text'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))


,oid,text,encoded
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...,"[[1, 5726, 7913, 336, 529, 856, 336, 7822, 228..."
1,452466036,Естественное восстановление после тяжелой трен...,"[[32703, 16969, 109, 21943, 4023, 6715, 12, 19..."
2,161038103,Тема нарядов продолжается Одна из британских ж...,"[[4987, 1, 4833, 1286, 276, 30777, 1, 1, 6052,..."
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...,"[[5146, 73746, 24, 1332, 41418, 676, 29, 992, ..."
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...,"[[19574, 29968, 9, 4569, 2923, 7340, 3583, 508..."


In [ ]:
test_ds = TestDataset(test_df['encoded'].values, test_df['oid'].values)

In [ ]:
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False)

In [ ]:
def predict(model, test_ds):
    model.eval()
    pred_cats = []
    pred_prob = []
    for oid, x, l in test_ds:
        x = x.long().to(device)
        l = torch.tensor(l).to(device)
        pred = model(x, l)
        sig = nn.Sigmoid()
        pred = sig(pred)
        pred_cats.append(np.argmax(pred.cpu().detach().numpy()))
        pred_prob.append(np.max(pred.cpu().detach().numpy()))

    return pred_cats, pred_prob

In [ ]:
pred_cats, pred_prob = predict(model_fixed, test_ds)


In [ ]:
test_df_out = test_df.copy()
test_df_out['cat'] = pred_cats
test_df_out['prob'] = pred_prob
test_df_out

,oid,text,encoded,cat,prob
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...,"[[1, 5726, 7913, 336, 529, 856, 336, 7822, 228...",5,0.994802
1,452466036,Естественное восстановление после тяжелой трен...,"[[32703, 16969, 109, 21943, 4023, 6715, 12, 19...",10,0.991776
2,161038103,Тема нарядов продолжается Одна из британских ж...,"[[4987, 1, 4833, 1286, 276, 30777, 1, 1, 6052,...",9,0.989645
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...,"[[5146, 73746, 24, 1332, 41418, 676, 29, 992, ...",5,0.906853
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...,"[[19574, 29968, 9, 4569, 2923, 7340, 3583, 508...",9,0.990521
...,...,...,...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...,"[[24101, 88160, 34023, 81, 2357, 2089, 35363, ...",3,0.997909
26256,279369911,Волейбол от первого лица Егора Пупынина переко...,"[[3021, 533, 2315, 1113, 17274, 102658, 93131,...",12,0.998766
26257,600699419,Вы были когда нибудь на свидании где вам задав...,"[[537, 328, 1088, 4469, 42, 68344, 1959, 454, ...",3,0.996058
26258,560223506,ТОП 20 самых эффективных общефизических упражн...,"[[1215, 12, 815, 36141, 1, 6061, 223, 39110, 1...",12,0.681270


In [ ]:
test_df_out = test_df_out[test_df_out.prob > 0.995]

In [ ]:
labels_inv = {v: k for k, v in labels.items()}
labels_inv

{0: 'winter_sport',
 1: 'extreme',
 2: 'football',
 3: 'boardgames',
 4: 'hockey',
 5: 'esport',
 6: 'athletics',
 7: 'motosport',
 8: 'basketball',
 9: 'tennis',
 10: 'autosport',
 11: 'martial_arts',
 12: 'volleyball'}

In [ ]:
test_df_out['category'] = test_df_out['cat'].map(labels_inv.get)
test_df_out

<ipython-input-184-0e71bf229de7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_out['category'] = test_df_out['cat'].map(labels_inv.get)


,oid,text,encoded,cat,prob,category
5,332983824,19 июня ABL разыграет 100 000 рублей и чемпион...,"[[12, 953, 94422, 63737, 4885, 3596, 15, 19673...",8,0.999872,basketball
6,306441436,Уже сейчас ты можешь оформить предзаказ фирмен...,"[[108, 503, 1332, 7875, 4481, 4539, 70967, 127...",10,0.999419,autosport
7,770457906,В Уфе пройдет турнир Мемориал Мустафина 25 и 2...,"[[9, 17132, 2186, 1634, 2932, 36678, 12, 15, 1...",12,0.999488,volleyball
8,720955404,Школа шахмат tokenoid tokenoid представляет ур...,"[[19519, 19520, 156, 156, 3040, 11162, 19520, ...",3,0.997642,boardgames
11,357676832,BLAST возвращается в Португалию 33 Многие ждал...,"[[6940, 6293, 9, 11262, 12, 3562, 1386, 299, 2...",5,0.997090,esport
...,...,...,...,...,...,...
26253,164938627,ФУНКЦИОНАЛЬНОЕ ТРЕНИРОВОЧНОЕ ЗАНЯТИЕ НА РАЗВИТ...,"[[51596, 46334, 10974, 42, 4139, 20382, 1899, ...",6,0.996600,athletics
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...,"[[24101, 88160, 34023, 81, 2357, 2089, 35363, ...",3,0.997909,boardgames
26256,279369911,Волейбол от первого лица Егора Пупынина переко...,"[[3021, 533, 2315, 1113, 17274, 102658, 93131,...",12,0.998766,volleyball
26257,600699419,Вы были когда нибудь на свидании где вам задав...,"[[537, 328, 1088, 4469, 42, 68344, 1959, 454, ...",3,0.996058,boardgames


In [ ]:
test_df_out[['oid', 'category']].to_csv('lstm1.csv', index=False)